In [1]:
import random
import os
import numpy as np
from os import listdir, remove, mkdir
from os.path import join as join_path

## Preprocessing europarl data

In [2]:
euro_data_dir = "../europarl/aligned/"
output_data_dir = "./data/"

### Join Files

In [3]:
for l_pair in listdir(euro_data_dir):
    pair_path = join_path(euro_data_dir, l_pair)
    for l_dir in listdir(pair_path):
        out_file = open(output_data_dir + l_pair + "_" + l_dir + ".txt", "w")
        l_path = join_path(pair_path, l_dir)
        for file_name in sorted(listdir(l_path)):
            # Consume content and split lines
            content = open(join_path(l_path, file_name), "r").read() 

            # Write content in output file
            out_file.write(content)


### Load corpus in memory

In [4]:
corpus = {}
for file_name in listdir(output_data_dir):
    path = join_path(output_data_dir, file_name)
    if os.path.isfile(path):
        file = open(path, "r+")
        l_pair = file_name[:5]
        if l_pair not in corpus:
            corpus[l_pair] = {}
        corpus[l_pair][file_name] = np.array(file.read().split('\n'), dtype=object)
        file.close()
        remove(path)

### Delete empty lines and lines with xml tags

In [5]:
for pair, l_data in corpus.items():

    # Find special lines
    empty_lines = np.array([], dtype=int)
    for _, value in l_data.items():
        is_empty = np.vectorize(lambda x: len(x) == 0 or x[0] == "<")
        indices = np.nonzero(is_empty(value))[0]
        empty_lines = np.concatenate((empty_lines, indices))
    empty_lines = np.unique(empty_lines)

    # Remove lines with an xml tag or an empty char
    for key, value in l_data.items():
        new_corpus = np.delete(value, empty_lines)
        corpus[pair][key] = new_corpus.tolist()
        print(len(new_corpus))
        

2048549
2048549
1957802
1957802
2007131
2007131


### Write new file(s)

In [8]:
for lang_pair, pair_dict in corpus.items():
    for file_name, data in pair_dict.items():
        file = open(join_path(output_data_dir, file_name), 'w')
        file.write("\n".join(data))
